<a href="https://colab.research.google.com/github/MittalNeha/AI-ML_classProjects/blob/master/Github_Diffusion_Training_Class_Conditional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Train a Class Conditional Image Generation using Diffusion Models:

- To see more details on the code and model training visit the [Github Repo](https://github.com/apapiu/guided-diffusion-keras)

Make sure to go to Runtime and set your Hardware -> Change Runtime Type and have the accelerator set to GPU!

Note that this notebook does **not** read or write anything to your machine nor does it need any access to google drive or outside sources. All the downloads happen within this notebook and once you close it, it will all go gone poof!

- Author: Alexandru Papiu
- Email: alexpapiu@gmail.com


In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2022 Alexandru Papiu

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
#clone the repo so you can mess around with it:
!git clone https://github.com/apapiu/guided-diffusion-keras.git
! pip install git+https://github.com/openai/CLIP.git

# #this is where the data is:
# !git lfs install
# !git clone https://huggingface.co/apapiu/diffusion_model_aesthetic_keras

import os
import sys
import numpy as np
import pandas as pd

from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10, cifar100
from keras.utils.vis_utils import plot_model

from tensorflow import keras
from matplotlib import pyplot as plt

####local imports:
sys.path.append("/content/guided-diffusion-keras/guided_diffusion")
from denoiser import get_network
from utils import batch_generator, plot_images, get_data, preprocess
from diffuser import Diffuser

In [ ]:
#########
# CONFIG:
#########

train_model=True
epochs = 50

file_name = "fashion_mnist" #cifar100, fashion_mnist, mnist, cifar10

# architecture
channels = 64
channel_multiplier = [1, 2, 3]
block_depth = 2
emb_size = 512  # CLIP/label embedding
num_classes = 12  # only used if precomputed_embedding=False
attention_levels = [0, 1, 0]

embedding_dims = 32
embedding_max_frequency = 1000.0

precomputed_embedding = False
save_in_drive = False
widths = [c * channels for c in channel_multiplier]

###train process config:
batch_size = 64
num_imgs = 100 #num imgs to test on - should be a square - 25, 64, 100 etc.
row = int(np.sqrt(num_imgs))
class_guidance = 3

learning_rate = 0.0003

MODEL_NAME = "model_test"
from_scratch = True #if False will load model from model path and continue training

data_dir = '/content/diffusion_model_aesthetic_keras'
captions_path = os.path.join(data_dir, "captions.csv")
imgs_path = os.path.join(data_dir, "imgs.npy")
embedding_path = os.path.join(data_dir, "embeddings.npy")

if save_in_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/'
    home_dir = os.path.join(drive_path, MODEL_NAME)
else:
    home_dir = MODEL_NAME

if not os.path.exists(home_dir):
    os.mkdir(home_dir)

model_path = os.path.join(home_dir, MODEL_NAME + ".h5")

In [ ]:
##################################
###########Loading Data And Model:
##################################

if file_name == "cifar10":
    (train_data, train_label_embeddings), (_, _) = cifar10.load_data()
    #0 is used as a unconditional embedding:
    train_label_embeddings = train_label_embeddings + 1
elif file_name == "cifar100":
    (train_data, train_label_embeddings), (_, _) = cifar100.load_data()
    #0 is used as unconditional embedding:
    train_label_embeddings = train_label_embeddings + 1
elif file_name == "fashion_mnist":
    (train_data, train_label_embeddings), (_, _) = fashion_mnist.load_data()
    train_data = train_data[:, :, :, None] #add extra dim at the end
    train_label_embeddings = train_label_embeddings[:, None]
    train_label_embeddings = train_label_embeddings + 1
    class_guidance = 1 #FASHION mnist does not do well with larger guidances - why?
elif file_name == "mnist":
    (train_data, train_label_embeddings), (_, _) = mnist.load_data()
    train_data = train_data[:, :, :, None] #add extra dim at the end
    train_label_embeddings = train_label_embeddings[:, None]
    train_label_embeddings = train_label_embeddings + 1
    class_guidance = 1.5 
else:
    #load the data from a numpy file file:
    captions = pd.read_csv(captions_path)
    train_data, train_label_embeddings = np.load(imgs_path), np.load(embedding_path)

    #train_data, train_label_embeddings = get_data(npz_file_name=file_name, prop=0.6, captions=False)

print(train_data.shape)

image_size = train_data.shape[1]
num_channels = train_data.shape[-1]

if precomputed_embedding:
    labels = train_label_embeddings[:num_imgs]
else:
    labels = np.array([[i] * row for i in np.arange(row)]).flatten()[:, None]
    labels += 1


np.random.seed(100)
rand_image = np.random.normal(0, 1, (num_imgs, image_size, image_size, num_channels))

if from_scratch:
    autoencoder = get_network(image_size,
                              widths,
                              block_depth,
                              num_classes=num_classes,
                              attention_levels=attention_levels,
                              emb_size=emb_size,
                              num_channels=num_channels,
                              precomputed_embedding=precomputed_embedding)

    autoencoder.compile(optimizer="adam", loss="mae")
else:
    autoencoder = keras.models.load_model(model_path)


##################
#Some data checks:
##################

print("Number of pamaters is {0}".format(autoencoder.count_params()))
pd.Series(train_data[:1000].ravel()).hist(bins=80)
plt.show()
print("Original Images:")
plot_images(preprocess(train_data[:num_imgs]), nrows=int(np.sqrt(num_imgs)))
plot_model(autoencoder, to_file=os.path.join(home_dir, 'model_plot.png'),
           show_shapes=True, show_layer_names=True)
print("Generating Images below:")

#############################
#!create generator and train:
#############################


diffuser = Diffuser(autoencoder,
                    class_guidance=class_guidance,
                    diffusion_steps=35)

if train_model:
    train_generator = batch_generator(autoencoder,
                                    model_path,
                                    train_data,
                                    train_label_embeddings,
                                    epochs,
                                    batch_size,
                                    rand_image,
                                    labels,
                                    home_dir,
                                    diffuser)

    autoencoder.optimizer.learning_rate.assign(learning_rate)

    eval_nums = autoencoder.fit(
        x=train_generator,
        epochs=epochs,
    )

Permutation Tests

In [ ]:
################################################

In [ ]:
# big_diffuser = Diffuser(autoencoder,
#                         class_guidance=3,
#                         diffusion_steps=50)

# label_num = 3

# labels_ohe[:] = label_num
# imgs = big_diffuser.reverse_diffusion(rand_image, labels_ohe)
# imgs_combined = np.vstack([imgs[:50], 
#                            preprocess(train_data[train_label_embeddings[:, 0] == label_num][:50])]
#                                       )
# plot_images(imgs_combined, nrows=10)
# plt.show()
# print("permutation_test")
# np.random.shuffle(imgs_combined)
# plot_images(imgs_combined, nrows=10)
# plt.show()